In [54]:
# Install required packages.
import os
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


In [55]:
data_dir = 'c:/Users/david/MT_data/extracted_patches/mutant_graphs_classification/'

# Show an example Graph

In [56]:
path = os.path.join(data_dir, 'AAAH_GraphPatch.pkl')

In [57]:
from f_helper_functions import *

patch = load_object(path)
print(patch.edge_index.shape)
print(patch.edge_weight.shape)
print(patch.fitness)
print(patch.A.shape)
print(patch.mutant)
print()
print(patch)

(2, 5492)
(5492, 1)
0
(1073, 1073)
AAAH

            Number of Nodes: 1073
            Features: (1073, 16)
            Adjacency Matrix: (1073, 1073)
            Edge Weights (Geodesic Distances): (5492, 1)
            Edge Index: (2, 5492)
            Fitness: 0
            Coordinates of Points: (1073, 3)
            Mutant Name: AAAH


## Define customized Dataset

In [58]:
#from torch.utils.data import Dataset
from c_PatchDataset_GNN import PatchDataset

batch_size = 8
dataset = PatchDataset(data_dir = data_dir)
len(dataset)

1500

In [59]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')


data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of node features: {data.num_node_features}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains self-loops: {data.has_self_loops()}')


Dataset: PatchDataset(1500):
Number of graphs: 1500

Data(x=[1049, 16], edge_index=[2, 5386], edge_attr=[5386, 1], y=1, pos=[1049, 3])
Number of nodes: 1049
Number of node features: 16
Number of edges: 5386
Average node degree: 5.13
Contains self-loops: False


In [60]:
data.edge_attr

tensor([[0.3675],
        [0.6447],
        [0.6881],
        ...,
        [1.0733],
        [1.0933],
        [1.2138]], dtype=torch.float64)

In [61]:
len(dataset)

1500

In [62]:
n_train = int(len(dataset)*0.8)
n_test = len(dataset) - n_train 

trainset, testset = torch.utils.data.random_split(dataset, [n_train, n_test])

print(f'Number of training graphs: {len(trainset)}')
print(f'Number of test graphs: {len(testset)}')

Number of training graphs: 1200
Number of test graphs: 300


In [63]:
from torch_geometric.loader import DataLoader

trainloader = DataLoader(dataset = trainset, batch_size= batch_size, shuffle = True)
testloader = DataLoader(dataset = testset, batch_size= batch_size, shuffle = True)

## Training a Graph Neural Network (GNN)

Training a GNN for graph classification usually follows a simple recipe:

1. Embed each node by performing multiple rounds of message passing
2. Aggregate node embeddings into a unified graph embedding (**readout layer**)
3. Train a final classifier on the graph embedding

There exists multiple **readout layers** in literature, but the most common one is to simply take the average of node embeddings:

$$
\mathbf{x}_{\mathcal{G}} = \frac{1}{|\mathcal{V}|} \sum_{v \in \mathcal{V}} \mathcal{x}^{(L)}_v
$$

PyTorch Geometric provides this functionality via [`torch_geometric.nn.global_mean_pool`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.glob.global_mean_pool), which takes in the node embeddings of all nodes in the mini-batch and the assignment vector `batch` to compute a graph embedding of size `[batch_size, hidden_channels]` for each graph in the batch.

The final architecture for applying GNNs to the task of graph classification then looks as follows and allows for complete end-to-end training:

In [64]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_max_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(16, hidden_channels)                   # 16-->64 node features
        self.conv2 = GCNConv(hidden_channels, hidden_channels)      # 64-->64 node features
        self.conv3 = GCNConv(hidden_channels, hidden_channels)      # 64-->64 node features
        self.lin = Linear(hidden_channels, 2)                       # 64-->1 node features fully connected layer

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        # Take the sum over all nodes in each graph = 16 values per graph
        x = global_max_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(16, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


Here, we again make use of the [`GCNConv`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv) with $\mathrm{ReLU}(x) = \max(x, 0)$ activation for obtaining localized node embeddings, before we apply our final classifier on top of a graph readout layer.

Let's train our network for a few epochs to see how well it performs on the training as well as test set:

In [65]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    for data in trainloader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.


def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 10):
    loss = train()
    
    train_acc = test(trainloader)
    test_acc = test(testloader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.7167, Test Acc: 0.6767
Epoch: 002, Train Acc: 0.7167, Test Acc: 0.6767
Epoch: 003, Train Acc: 0.7158, Test Acc: 0.6767
Epoch: 004, Train Acc: 0.2833, Test Acc: 0.3267
Epoch: 005, Train Acc: 0.7208, Test Acc: 0.6867


KeyboardInterrupt: 